In [ ]:
import argparse
import os
import glob
import time
from pathlib import Path
import onnxruntime_genai as og

In [ ]:
# Download model from HF
model_path = "gpu/gpu-int4-rtn-block-32"

In [8]:
execution_provider = 'cpu'
image_paths = glob.glob("../BarCodeTMNA/TestingOCR/*.JPG")
prompt = "Enter the path to the image you want to test: "

In [10]:
config = og.Config(model_path)
config.clear_providers()
if execution_provider != "cpu":
    print(f"Setting model to {execution_provider}...")
    config.append_provider(execution_provider)
model = og.Model(config)
print("Model loaded")

Model loaded


In [11]:
processor = model.create_multimodal_processor()
tokenizer_stream = processor.create_stream()

In [23]:
image_path = image_paths[0]

In [24]:
# Load the image
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Image file not found: {image_path}")
print(f"Using image: {image_path}")
images = og.Images.open(image_path)

Using image: ../BarCodeTMNA/TestingOCR\d170_d3549_snp_serial_number_photo.jpg


In [26]:
prompt_text = "What is shown in this image?"  # Specify your prompt here

In [27]:
# Construct the prompt with the image tag
prompt = f"<|user|>\n<|image_1|>\n{prompt_text}<|end|>\n<|assistant|>\n"
print("Processing images and prompt...")

Processing images and prompt...


In [28]:
inputs = processor(prompt, images=images)

In [32]:
# Generate a response
print("Generating response...")
params = og.GeneratorParams(model)
params.set_inputs(inputs)
params.set_search_options(max_length=7680)

generator = og.Generator(model, params)
start_time = time.time()

# Generate the response token by token
while not generator.is_done():
    generator.compute_logits()
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end="", flush=True)

print()
total_run_time = time.time() - start_time
print(f"Total Time: {total_run_time:.2f} seconds")

Generating response...
The image shows a close-up of a barcode with the text 'X RND0271282' visible on it. The barcode is partially occluded, and the rest of the text is not clear.
Total Time: 252.52 seconds
